##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar métricas e otimizadores

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/metrics_optimizers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/metrics_optimizers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

No TF1, `tf.metrics` é o namespace da API para todas as funções de métrica. Cada uma das métricas é uma função que usa `label` e `prediction` como parâmetros de entrada e retorna o tensor da métrica correspondente como resultado. No TF2, `tf.keras.metrics` contém todas as funções e objetos de métrica. O objeto `Metric` pode ser usado com `tf.keras.Model` e `tf.keras.layers.layer` para calcular valores de métrica.

## Configuração

Vamos começar com algumas importações necessárias do TensorFlow,

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

e preparar alguns dados simples para demonstração:

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [0, 0, 1]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [0, 1, 1]

## TF1: tf.compat.v1.metrics com Estimator

No TF1, as métricas podem ser adicionadas a `EstimatorSpec` como `eval_metric_ops`, e o op é gerado por meio de todas as funções de métricas definidas em `tf.metrics`. Você pode acompanhar o exemplo para ver como usar `tf.metrics.accuracy`.

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(2)(features)
  predictions = tf.math.argmax(input=logits, axis=1)
  loss = tf1.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  accuracy = tf1.metrics.accuracy(labels=labels, predictions=predictions)
  return tf1.estimator.EstimatorSpec(mode, 
                                     predictions=predictions,
                                     loss=loss, 
                                     train_op=train_op,
                                     eval_metric_ops={'accuracy': accuracy})

estimator = tf1.estimator.Estimator(model_fn=_model_fn)
estimator.train(_input_fn)

In [ ]:
estimator.evaluate(_eval_input_fn)

Além disso, as métricas podem ser adicionadas ao estimador diretamente via `tf.estimator.add_metrics()`.

In [ ]:
def mean_squared_error(labels, predictions):
  labels = tf.cast(labels, predictions.dtype)
  return {"mean_squared_error": 
          tf1.metrics.mean_squared_error(labels=labels, predictions=predictions)}

estimator = tf1.estimator.add_metrics(estimator, mean_squared_error)
estimator.evaluate(_eval_input_fn)

## TF2: API Keras Metrics com tf.keras.Model

No TF2, `tf.keras.metrics` contém todas as classes e funções de métricas. Elas são projetadas em estilo OOP e se integram intimamente com outras APIs `tf.keras`. Todas as métricas podem ser encontradas no namespace `tf.keras.metrics` e geralmente há um mapeamento direto entre `tf.compat.v1.metrics` com `tf.keras.metrics`.

No exemplo a seguir, as métricas são adicionadas no método `model.compile()`. Os usuários precisam apenas criar a instância da métrica, sem especificar o rótulo e o tensor de previsão. O modelo Keras encaminhará a saída e o rótulo do modelo para o objeto de métricas.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

inputs = tf.keras.Input((2,))
logits = tf.keras.layers.Dense(2)(inputs)
predictions = tf.math.argmax(input=logits, axis=1)
model = tf.keras.models.Model(inputs, predictions)
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer, loss='mse', metrics=[tf.keras.metrics.Accuracy()])

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

Com a execução antecipada (eager execution) ativada, as instâncias `tf.keras.metrics.Metric` podem ser usadas diretamente para avaliar dados numpy ou tensores eager. Os objetos `tf.keras.metrics.Metric` são containers stateful. O valor da métrica pode ser atualizado via `metric.update_state(y_true, y_pred)`, e o resultado pode ser recuperado com `metrics.result()`.


In [ ]:
accuracy = tf.keras.metrics.Accuracy()

accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[0, 0, 0, 1])
accuracy.result().numpy()


In [ ]:
accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[0, 0, 0, 0])
accuracy.update_state(y_true=[0, 0, 1, 1], y_pred=[1, 1, 0, 0])
accuracy.result().numpy()

Para mais detalhes sobre `tf.keras.metrics.Metric`, consulte a documentação da API em `tf.keras.metrics.Metric`, assim como o [guia de migração](https://www.tensorflow.org/guide/effective_tf2#new-style_metrics_and_losses).

## Migração de otimizadores TF1.x para otimizadores Keras

Os otimizadores em `tf.compat.v1.train`, como o [otimizador Adam](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/AdamOptimizer) e o [otimizador de método do gradiente](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/GradientDescentOptimizer), têm equivalentes em `tf.keras.optimizers`.

A tabela abaixo resume como você pode converter esses otimizadores herdados aos seus equivalentes Keras. Você pode substituir diretamente a versão TF1.x pela versão TF2, a menos que passos adicionais (como [atualizar a taxa de aprendizado padrão](../../guide/effective_tf2.ipynb#optimizer_defaults)) sejam obrigatórios.

Observe que a conversão de seus otimizadores [pode deixar os checkpoints antigos incompatíveis](./migrating_checkpoints.ipynb).

<table>
  <tr>
    <th>TF1.x</th>
    <th>TF2</th>
    <th>Passos adicionais</th>
  </tr>
  <tr>
    <td>`tf.v1.train.GradientDescentOptimizer`</td>
    <td>`tf.keras.optimizers.SGD`</td>
    <td>Nenhum</td>
  </tr>
  <tr>
    <td>`tf.v1.train.MomentumOptimizer`</td>
    <td>`tf.keras.optimizers.SGD`</td>
    <td>Inclua o argumento `momentum`</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdamOptimizer`</td>
    <td>`tf.keras.optimizers.Adam`</td>
    <td>Renomeie os argumentos `beta1` e `beta2` para `beta_1` e `beta_2`</td>
  </tr>
  <tr>
    <td>`tf.v1.train.RMSPropOptimizer`</td>
    <td>`tf.keras.optimizers.RMSprop`</td>
    <td>Renomeie o argumento `decay` para `rho`</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdadeltaOptimizer`</td>
    <td>`tf.keras.optimizers.Adadelta`</td>
    <td>Nenhum</td>
  </tr>
  <tr>
    <td>`tf.v1.train.AdagradOptimizer`</td>
    <td>`tf.keras.optimizers.Adagrad`</td>
    <td>Nenhum</td>
  </tr>
  <tr>
    <td>`tf.v1.train.FtrlOptimizer`</td>
    <td>`tf.keras.optimizers.Ftrl`</td>
    <td>Remova os argumentos `accum_name` e `linear_name`</td>
  </tr>
  <tr>
    <td>`tf.contrib.AdamaxOptimizer`</td>
    <td>`tf.keras.optimizers.Adamax`</td>
    <td>Renomeie os argumentos `beta1` e `beta2` para `beta_1` e `beta_2`</td>
  </tr>
  <tr>
    <td>`tf.contrib.Nadam`</td>
    <td>`tf.keras.optimizers.Nadam`</td>
    <td>Renomeie os argumentos `beta1` e `beta2` para `beta_1` e `beta_2`</td>
  </tr>
</table>

Observação: No TF2, todos os epsilons (constantes de estabilidade numérica) agora são padronizados para `1e-7` em vez de `1e-8`. Essa diferença é insignificante na maioria dos casos de uso.